# Multi-Step Tool Use

Tool use allows developers to connect Cohere's models to external tools like search engines, APIs, functions, databases, etc.

Multi-step tool use is an extension of this basic idea, and allows the model to call more than one tool in a sequence of steps, using the results from one tool call in a subsequent step. This process allows the language model to reason, perform dynamic actions, and quickly adapt on the basis of information coming from external sources.

The recommended way to achieve [multi-step tool use with Cohere](https://docs.cohere.com/docs/multi-step-tool-use) is by leveraging the [Langchain framework](https://python.langchain.com/docs/integrations/providers/cohere#react-agent) in Python.

## Install Dependencies

In [1]:
! pip install --quiet langchain langchain_cohere langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.9/145.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# LLM
import os
os.environ['COHERE_API_KEY'] = <your-api-key>

## Define tools

Your agent will be equipped with the following tools. The model can pick between

- doing a search on the web
- and/or retrieving data from a vector store
- and/or using a python interpreter
- and/or directly answering [ this tool comes out of the box! ]

Plus the model can self-reflect.

#### Web search
You can easily equip your agent with web search!

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ["TAVILY_API_KEY"] = <your-api-key> # you can create an API key for free on Tavily's website

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


from langchain_core.pydantic_v1 import BaseModel, Field
class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")
internet_search.args_schema = TavilySearchInput

#### Vector store
You can easily equip your agent with a vector store!

In [4]:
!pip --quiet install faiss-cpu tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.3 MB/s eta 0:00:00


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings

# Set embeddings
embd = CohereEmbeddings()

# Docs to index
urls = [
    "https://paulgraham.com/best.html",
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = FAISS.from_documents(
    documents=doc_splits,
    embedding=embd,
)

vectorstore_retriever = vectorstore.as_retriever()


In [6]:
from langchain.tools.retriever import create_retriever_tool

vectorstore_search = create_retriever_tool(
    retriever=vectorstore_retriever,
    name="vectorstore_search",
    description="Retrieve relevant info from a vectorstore that contains information from Paul Graham about how to write good essays."
)

#### Python interpreter tool
You can easily equip your agent with a python interpreter!

In [7]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
python_tool.name = "python_interpreter"

# from langchain_core.pydantic_v1 import BaseModel, Field
class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
python_tool.args_schema = ToolInput

#### Transform any Python function in a Tool
You can easily equip your agent with any Python function!

In [8]:
from langchain_core.tools import tool
import random

@tool
def random_operation_tool(a: int, b: int):
  """Calculates a random operation between the inputs."""
  coin_toss = random.uniform(0, 1)
  if coin_toss > 0.5:
    return {'output': a*b}
  else:
    return {'output': a+b}

random_operation_tool.name = "random_operation" # use python case
random_operation_tool.description = "Calculates a random operation between the inputs."

from langchain_core.pydantic_v1 import BaseModel, Field
class random_operation_inputs(BaseModel):
    a: int = Field(description="First input")
    b: int = Field(description="Second input")
random_operation_tool.args_schema = random_operation_inputs

## Create ReAct Agent

The model can smartly pick the right tool(s) for the user query, call them in any sequence, analyze the results and self-reflect.  
Once the model considers it has enough information to answer the user question, it generates the final answer.

In [13]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.chat_models import ChatCohere

# LLM
llm = ChatCohere(model="command-r-plus", temperature=0.3)

# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource. You are equipped with an internet search tool and a special vectorstore of information about how to write good essays.
You also have a 'random_operation_tool' tool, you must use it to compute the random operation between two numbers.
"""


# Prompt template
prompt = ChatPromptTemplate.from_template("{input}")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=llm,
    tools=[internet_search, vectorstore_search, python_tool, random_operation_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search, vectorstore_search, python_tool, random_operation_tool], verbose=True)

## Ask a standalone question to the ReAct agent
A question that requires using a predefined tool from Langchain

In [14]:
response = agent_executor.invoke({
    "input": "I want to write an essay about the Roman Empire. Any tips for writing an essay? Any fun facts?",
    "preamble": preamble,
})

response['output']

# note that the model smartly looks in the vector db, and then online



> Entering new AgentExecutor chain...

I will search for tips on writing an essay and fun facts about the Roman Empire.
{'tool_name': 'vectorstore_search', 'parameters': {'query': 'tips for writing an essay'}}
be? I should have asked how do you write essays well? Though
these seem only phrasing apart, their answers diverge. The answer
to the first question, as we've seen, isn't really about essay
writing. The second question forces it to be.Writing essays, at its best, is a way of discovering ideas. How do
you do that well? How do you discover by writing?An essay should ordinarily start with what I'm going to call a
question, though I mean this in a very general sense: it doesn't
have to be a question grammatically, just something that acts like
one in the sense that it spurs some response.How do you get this initial question? It probably won't work to
choose some important-sounding topic at random and go at it.
Professional traders won't even trade unless they have what they
call an

"Here are some tips for writing an essay:\n- Start with a question that spurs some response.\n- Don't choose a topic at random, make sure you have a way in, a new insight or approach.\n- You don't need a complete thesis, just a gap to explore.\n- You can get ideas by talking to people, reading, doing and building things, and going places and seeing things.\n- You can improve the quality of your ideas by increasing the breadth and depth of what goes in.\n- You can get breadth by reading and talking about a wide range of topics.\n- You can get depth by doing and having to solve problems.\n- You can also get ideas by talking to people who make you have new ideas.\n\nHere are some fun facts about the Roman Empire:\n- At its peak, the empire stretched from North Africa to Britain, reigning over 60 million people.\n- The story of Rome's warrior founder and the twins and their wolf step-mother was depicted on Rome's first coins.\n- The Crossing of the Rhine in 405/6 AD brought around 100,000 

A question that requires the large language model to use a custom tool.

In [22]:
response = agent_executor.invoke({
    "input": "Calculate the result of the random operation of 10 and 20. Then find a few fun facts about that number, as well as its prime factors.",
    "preamble": preamble,
})

response['output']

# note that the model uses a sequence of tools



> Entering new AgentExecutor chain...

First, I will calculate the result of the random operation between 10 and 20. Then, I will search for fun facts about that number and its prime factors.
{'tool_name': 'random_operation_tool', 'parameters': {'a': 10, 'b': 20}}
random_operation_tool is not a valid tool, try one of [internet_search, vectorstore_search, python_interpreter, random_operation].
I received an error message when trying to use the random_operation_tool. I will now try using the python_interpreter tool to calculate the random operation between 10 and 20.
{'tool_name': 'python_interpreter', 'parameters': {'code': "import random\n\n# Define the two numbers\na = 10\nb = 20\n\n# Calculate the random operation\nresult = random.choice(['+', '-', '*', '/'])\n\n# Perform the operation\nif result == '+':\n    answer = a + b\nelif result == '-':\n    answer = a - b\nelif result == '*':\n    answer = a * b\nelif result == '/':\n    answer = a / b\n\nprint(f'The result of the random o

"The result of the random operation is **200**. Here are some fun facts about the number 200:\n- It is the smallest base 10 unprimeable number, meaning it cannot be turned into a prime number by changing just one of its digits to any other digit.\n- According to Bullinger's study of biblical literature, the number 200 signifies 'insufficiency'.\n- The number 200 is an even composite number composed of two distinct primes.\n\nThe prime factors of 200 are 2 and 5."

A question that requires the large language model to directly answer.

In [23]:
response = agent_executor.invoke({
    "input": "Hey how are you?",
    "preamble": preamble,
})

response['output']

# note that the modle can directly answer!



> Entering new AgentExecutor chain...

I will respond to the user's greeting.
{'tool_name': 'directly_answer', 'parameters': {}}
directly_answer is not a valid tool, try one of [internet_search, vectorstore_search, python_interpreter, random_operation].Relevant Documents: None
Cited Documents: None
Answer: I'm an AI chatbot, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?
Grounded answer: I'm an AI chatbot, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?

> Finished chain.


"I'm an AI chatbot, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?"

## Ask a more complex question to the ReAct agent
A question that requires using multipe tools, in sequence

In [27]:
response = agent_executor.invoke({
    "input": "In what year was the company that was founded as Sound of Music went public? What was its stock price in 2000 and 2010.",
    "preamble": preamble,
})

response['output']



> Entering new AgentExecutor chain...

I will search for the company that was founded as Sound of Music. Then, I will search for the year it went public. Finally, I will search for its stock price in 2000 and 2010.
{'tool_name': 'internet_search', 'parameters': {'query': 'company founded as Sound of Music'}}
[{'url': 'https://www.mprnews.org/story/2012/05/15/best-buy-richard-schulze-legacy', 'content': 'Amazon is taking a piece of everyone\'s business,"\nSome analysts have questioned the extent to which Schulze\'s strong hand over the company has held it back in the face of digital competition. "\nSchulze hit on the "Best Buy" strategy when a twister ripped open a store and the company held a huge sale with low prices and lots of promotion. And Richard Schulz was able to outthink and outmaneuver the competition to put up Best Buy as the definitive place in the brick-and-mortar world to buy electronic equipment," Spector said.\n Best Buy\'s Richard Schulze: Stereo seller to retail gia

'Best Buy, which was founded as Sound of Music, went public in 1985. Its stock price in 2000 was between $5.93 and $16.24 and in 2010, it was between $20.27 and $31.24.'

## Have a multi-turn conversation with the ReAct agent
The chat history enables you to have multi-turn conversations with the ReAct agent.

In [28]:
# Step 1: Construct the chat history as a list of LangChain Messages, ending with the last user message
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="I'm considering switching to Oracle for my CRM."),
    AIMessage(content="That sounds like a good idea! How can I help you?"),
    HumanMessage(content="Recap me all the info you can find about their offering."),
]

prompt = ChatPromptTemplate.from_messages(chat_history)

In [29]:
# Step 2: When you make the agent, specify the chat_history as the prompt, e.g.
# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=llm,
    tools=[internet_search, vectorstore_search, python_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search, vectorstore_search, python_tool], verbose=True)

In [30]:
# Step 3: When you invoke the agent_executor there's no need to pass anything else into invoke
response = agent_executor.invoke({
    "preamble": preamble,
})

response['output']



> Entering new AgentExecutor chain...

I will search for 'Oracle CRM offering' and relay the information I find to the user.
{'tool_name': 'internet_search', 'parameters': {'query': 'Oracle CRM offering'}}
[{'url': 'https://docs.oracle.com/en/applications/siebel/', 'content': 'Siebel CRM delivers a combination of transactional, analytical, and engagement features to manage all customer-facing operations. With solutions tailored to more than 20 industries, Siebel CRM delivers comprehensive on-premise CRM solutions that are tailored industry solutions with role-based customer intelligence and pre-built integration.'}, {'url': 'https://www.softwareadvice.com/resources/breaking-down-oracle-crm/', 'content': "Oracle's Marketing Cloud provides a comprehensive set of tools that cover a range of digital marketing needs. This includes tools for cross-channel marketing, i.e., marketing automation for both B2B and B2C marketers, as well as data management, content marketing and social media mar

"Oracle's CRM offering includes the following:\n- Marketing Cloud, which provides a comprehensive set of tools that cover a range of digital marketing needs, including cross-channel marketing, marketing automation, data management, content marketing and social media marketing\n- CRM On Demand, which is a full-featured marketing automation product with features from lead management and nurturing, to measuring marketing ROI\n- Siebel CRM, which delivers a combination of transactional, analytical, and engagement features to manage all customer-facing operations, with solutions tailored to more than 20 industries\n- Siebel CRM Life Sciences, which provides personalised content delivery tools to help sales and marketing teams deliver the right messages during each customer interaction\n- Oracle Client Experience (CX), a connected suite of tools that transcends standard CRM"